In [1]:
import pandas as pd
import numpy as np
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO  

In [6]:
f="sbi.pdf"

def tabu(f):
    tables=tabula.read_pdf(f,
                          lattice=True,
                          pages="all",
                          silent=True,
                          multiple_tables=True,
                          pandas_options={'header':None})
    df=pd.DataFrame()
    df=pd.concat([c for c in tables]).drop_duplicates()
    return df

def proceed(df):
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            print("\nColumn Headers Missing"); pass
    try:
        df = df.drop(["Init."], axis=1)
    except:
        try:
            df = df.drop(["Branch Name"], axis=1)
        except:pass

    try:
        df = df[~df.index.isin(df[df.apply(lambda row:row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(),axis=1)==True].index)]
        #df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False)==1)].index,axis=1,inplace=True)
    except:
        pass
    df.to_csv("parsed.csv",index=0)
    return df


def fpage(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 1
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,caching=caching, check_extractable=False):
        interpreter.process_page(page)

    text = retstr.getvalue()
    #return text.lstrip().split("\n")

    ks=[c.upper() for c in text.lstrip().split("\n") [:50] if "" !=c ]
    print(ks)
    
    lst=[]
    try:
        for j in ks:
            if "TRANSACTION TYPE" in j:
                index_no = ks.index(j)
                for k in ks[0:index_no+1]:
                    if k not in lst:
                        lst.append(k)
                return lst
            elif "ACCOUNT STATEMENT FROM" in j:
                index_no = ks.index(j)
                for k in ks[0:index_no+1]:
                    if k not in lst:
                        lst.append(k)
                return lst
                
    except:
        pass                 

cst_info = fpage(f)

df = tabu(f); df = proceed(df)

['ACCOUNT NAME                   :(CID:9)PERSONAL AIR QUALITY SYSTEMS PVT LTD', 'ADDRESS', 'NO- 306, VISHAL NEST, AMRUTAHALLI  TALA CAUVERY', 'LAYOUT  BANGALORE  BANGALORE', 'BANGALORE', 'KARNATAKA-560092', 'INDIA', 'DATE                                    :(CID:9)24 JAN 2019', 'ACCOUNT NUMBER                 :(CID:9)00000033561591501', 'ACCOUNT DESCRIPTION            :(CID:9)CA-GEN-PUB OTH-NONRURAL-INR', 'BRANCH                                :(CID:9)ELECTRONIC CITY', 'DRAWING POWER                   :(CID:9)0.00', 'INTEREST RATE(% P.A.)          :(CID:9)16.7', 'MOD BALANCE                     :(CID:9)0.00', 'CIF NO.                               :(CID:9)87126900468', 'IFS CODE                            :(CID:9)SBIN0009044', 'MICR CODE                        :(CID:9)560002065', 'BALANCE AS ON 1 JUL 2018  :(CID:9)3,809.59', ' ', 'ACCOUNT STATEMENT FROM 1 JUL 2018 TO 31 DEC 2018', 'TXN', 'DATE', 'DESCRIPTION REF', 'BRANCH', 'CODE', 'VALUE', 'DATE', '2 JUL', '2018', '2 JUL', '2018', '3 